In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import os
import pprint


In [6]:
load_dotenv(dotenv_path = "password.env") 

# MONGO_URI example: MONGO_URI=mongodb://localhost:27017/

mongo_uri = os.getenv("mongo_uri")
mongo_database = os.getenv("mongo_database")

client = MongoClient(mongo_uri)

db = client[mongo_database]

In [7]:
# Loading in csv data
patrons_data = pd.read_csv('patrons.csv')
inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
inventory['isbn'] = inventory['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')


# Converting to dictionaries
patrons_dict = patrons_data.to_dict('records')
loans_dict = loans.to_dict('records')
books_dict = books.to_dict('records')
inventory_dict = inventory.to_dict('records')
authors_dict = authors.to_dict('records')

# Adding authors as a subset of books
for item in books_dict:
    item['author'] = {}
for item in authors_dict:
    author_id = item['author_id']
    for book in books_dict:
        if book['author_id'] == author_id:
            book['author'] = item

# Merging books' meta data with inventoried books
for item in books_dict:
    isbn = item['isbn']
    for book in inventory_dict:
        if book['isbn'] == isbn:
            book['title'] = item['title']
            book['pub_year'] = item['pub_year']
            book['genre'] = item['genre']
            book['overdue_fee_rate'] = item['overdue_fee_rate']
            book['author'] = item['author']

# Adding inventories books as a subset of loans
for item in loans_dict:
    item['book'] = {}
for item in inventory_dict:
    book_id = item['book_id']
    for loan in loans_dict:
        if loan['book_id'] == book_id:
            loan['book'] = item
    item.pop('book_id')

# Inserting total overdue fees per patron
for patron in patrons_dict:
    patron['loan_history'] = []
    patron['total_fees_owed'] = 0
    for item in loans_dict:
        days = item['days_overdue']
        if pd.isna(days):
            days = 0
        rate = item['book']['overdue_fee_rate']
        fee = rate * days
        if item['patron_id'] == patron['patron_id']:
            patron['total_fees_owed'] += fee

# Adding loans as a subset of patrons
for item in loans_dict:
    patron_id = item['patron_id']
    for patron in patrons_dict:
        if patron['patron_id'] == patron_id:
            patron['loan_history'].append(item)
            item.pop("loan_id")
            item.pop("book_id")
            item.pop('patron_id')
            break


# Getting rid of `patron_id` variable in patrons
for item in patrons_dict:
    item.pop('patron_id', None)

patrons_dict[-4]


{'first_name': 'Chloe',
 'last_name': 'Robinson',
 'address': '305 Redwood Circle',
 'phone_number': '101-010-1010',
 'age': 29,
 'loan_history': [{'loan_date': '2024-12-27',
   'due_date': '2025-01-26',
   'return_date': '2025-01-29',
   'days_overdue': 3.0,
   'book': {'isbn': '9780374528379',
    'condition': 'Fair',
    'title': 'Interpreter of Maladies',
    'pub_year': 1999,
    'genre': 'Short Stories',
    'overdue_fee_rate': 0.25,
    'author': {'author_id': 19,
     'first_name': 'Jhumpa',
     'last_name': 'Lahiri',
     'nationality': 'Indian-American',
     'language': 'English',
     'birth_year': '1967',
     'biography': "Jhumpa Lahiri won the Pulitzer Prize for 'Interpreter of Maladies'."}}},
  {'loan_date': '2025-05-22',
   'due_date': '2025-06-21',
   'return_date': nan,
   'days_overdue': nan,
   'book': {'isbn': '9780812981602',
    'condition': 'Fair',
    'title': 'The Vegetarian',
    'pub_year': 2007,
    'genre': 'Literary Fiction',
    'overdue_fee_rate': 0.5

In [9]:
# PATRONS COLLECTION

# Adding patron objects to MongoDB
patrons = db.patrons
patrons.insert_many(patrons_dict)

# Delete current objects to repopulate if necessary
# patrons.delete_many({})

InsertManyResult([ObjectId('683763f18efeb56de1ad4ae0'), ObjectId('683763f18efeb56de1ad4ae1'), ObjectId('683763f18efeb56de1ad4ae2'), ObjectId('683763f18efeb56de1ad4ae3'), ObjectId('683763f18efeb56de1ad4ae4'), ObjectId('683763f18efeb56de1ad4ae5'), ObjectId('683763f18efeb56de1ad4ae6'), ObjectId('683763f18efeb56de1ad4ae7'), ObjectId('683763f18efeb56de1ad4ae8'), ObjectId('683763f18efeb56de1ad4ae9'), ObjectId('683763f18efeb56de1ad4aea'), ObjectId('683763f18efeb56de1ad4aeb'), ObjectId('683763f18efeb56de1ad4aec'), ObjectId('683763f18efeb56de1ad4aed'), ObjectId('683763f18efeb56de1ad4aee'), ObjectId('683763f18efeb56de1ad4aef'), ObjectId('683763f18efeb56de1ad4af0'), ObjectId('683763f18efeb56de1ad4af1'), ObjectId('683763f18efeb56de1ad4af2'), ObjectId('683763f18efeb56de1ad4af3'), ObjectId('683763f18efeb56de1ad4af4'), ObjectId('683763f18efeb56de1ad4af5'), ObjectId('683763f18efeb56de1ad4af6'), ObjectId('683763f18efeb56de1ad4af7'), ObjectId('683763f18efeb56de1ad4af8'), ObjectId('683763f18efeb56de1ad4a

In [11]:
# BOOKS COLLECTION

# Adding book objects to MongoDB
books = db.books
books.insert_many(inventory_dict)

# Delete current objects to repopulate if necessary
# books.delete_many({})

InsertManyResult([ObjectId('683763fa8efeb56de1ad4b01'), ObjectId('683763fa8efeb56de1ad4b02'), ObjectId('683763fa8efeb56de1ad4b03'), ObjectId('683763fa8efeb56de1ad4b04'), ObjectId('683763fa8efeb56de1ad4b05'), ObjectId('683763fa8efeb56de1ad4b06'), ObjectId('683763fa8efeb56de1ad4b07'), ObjectId('683763fa8efeb56de1ad4b08'), ObjectId('683763fa8efeb56de1ad4b09'), ObjectId('683763fa8efeb56de1ad4b0a'), ObjectId('683763fa8efeb56de1ad4b0b'), ObjectId('683763fa8efeb56de1ad4b0c'), ObjectId('683763fa8efeb56de1ad4b0d'), ObjectId('683763fa8efeb56de1ad4b0e'), ObjectId('683763fa8efeb56de1ad4b0f'), ObjectId('683763fa8efeb56de1ad4b10'), ObjectId('683763fa8efeb56de1ad4b11'), ObjectId('683763fa8efeb56de1ad4b12'), ObjectId('683763fa8efeb56de1ad4b13'), ObjectId('683763fa8efeb56de1ad4b14'), ObjectId('683763fa8efeb56de1ad4b15'), ObjectId('683763fa8efeb56de1ad4b16'), ObjectId('683763fa8efeb56de1ad4b17'), ObjectId('683763fa8efeb56de1ad4b18'), ObjectId('683763fa8efeb56de1ad4b19'), ObjectId('683763fa8efeb56de1ad4b

In [39]:
# Patrons with the highest library fees

query1 = {'total_fees_owed': {'$gte': 2}}
result = patrons.find(query1).sort('total_fees_owed', -1)

print("Patrons With the Highest Library Fees:")
for e in result:
    print(f"{e['first_name']} {e['last_name']}" + ":", "$" + f"{e['total_fees_owed']}")


Patrons With the Highest Library Fees:
Olivia Smith: $12.0
Grace Clark: $6.75
James Davis: $5.0
Alexander Moore: $2.75
Mia Hernandez: $2.5
Sophia Miller: $2.25
Noah Brown: $2.0
